In [1]:
#pip install gurobipy

In [2]:
from gurobipy import *
import gurobipy as gp
import pandas as pd
import numpy as np

In [3]:
c = 5

maxSpeedTruck = 15.6464
maxSpeedDrone = 22.352
SL = 40
SR = 30
E = 1440.0
M = 5313

# set of the serviceable customers for the drones
Cdrones = {2, 3,}

# distance matrix for the selected instance
distance = [
[     0,  7663, 11389, 15022, 11557,  7801,     0],
[  7663,     0, 13324, 20798, 15531, 15388,  7663],
[ 11389, 13324,     0, 10381, 22810, 12903, 11389],
[ 15022, 20798, 10381,     0, 23513, 10161, 15022],
[ 11557, 15531, 22810, 23513,     0, 13407, 11557],
[  7801, 15388, 12903, 10161, 13407,     0,  7801],
[     0,  7663, 11389, 15022, 11557,  7801,     0]
]

# travel times matrix for vehicles
tauTruck = [
[    0.0000,  612.2015,  909.8738, 1200.1163,  923.2955,  623.2264,    0.0000],
[  612.2015,  0.0000, 1064.4621, 1661.5643, 1240.7806, 1229.3563,  612.2015],
[  909.8738, 1064.4621,    0.0000,  829.3441, 1822.3042, 1030.8282,  909.8738],
[ 1200.1163, 1661.5643,  829.3441,    0.0000, 1878.4673,  811.7682, 1200.1163],
[  923.2955, 1240.7806, 1822.3042, 1878.4673,    0.0000, 1071.0930,  923.2955],
[  623.2264, 1229.3563, 1030.8282,  811.7682, 1071.0930,    0.0000,  623.2264],
[    0.0000,  612.2015,  909.8738, 1200.1163,  923.2955,  623.2264,    0.0000]
]

# travel times matrix for drones
tauDrone = [
[    0.0000,  428.5411,  636.9117,  840.0814,  646.3068,  436.2585,    0.0000],
[  428.5411,    0.0000,  745.1235, 1163.0950,  868.5464,  860.5494,  428.5411],
[  636.9117,  745.1235,    0.0000,  580.5409, 1275.6129,  721.5797,  636.9117],
[  840.0814, 1163.0950,  580.5409,    0.0000, 1314.9271,  568.2377,  840.0814],
[  646.3068,  868.5464, 1275.6129, 1314.9271,    0.0000,  749.7651,  646.3068],
[  436.2585,  860.5494,  721.5797,  568.2377,  749.7651,    0.0000,  436.2585],
[    0.0000,  428.5411,  636.9117,  840.0814,  646.3068,  436.2585,    0.0000]
]



In [4]:
# set of all the drone's tuples
F = {
    (0,1,2), (0,1,4), (0,1,5), (0,1,6), (0,2,1), (0,2,3), (0,2,5), (0,2,6), (0,3,2), (0,3,5), (0,5,1), (0,5,2), (0,5,3), (0,5,4),(0,5,6), (1,2,3), (1,2,6), (1,5,3),
(1,5,6), (2,1,6), (2,3,5), (2,3,6), (2,5,3), (2,5,6), (3,2,1), (3,2,5), (3,2,6), (3,5,1), (3,5,2), (3,5,4), (3,5,6), (4,1,6), (4,5,3), (4,5,6), (5,1,6), (5,2,3), (5,2,6),
(5,3,2), (5,3,6)
}
    

In [5]:
m = Model("FSTSP")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-07


In [6]:
# Decision variables
x = {}
for i in range(c+1):
    for j in range(1, c+2):
        if i != j:
            x[i,j] = m.addVar(vtype = GRB.BINARY, name = f'x{[i,j]}')

In [7]:
y = {}
for i in range(c+1):
    for j in Cdrones:
        for k in range(1, c+2):
            y[i,j,k] = m.addVar(vtype = GRB.BINARY, name = f'y{[i,j,k]}')

In [8]:
t_T = {}
for j in range(c+2):
    t_T[j] = m.addVar(vtype = GRB.CONTINUOUS, name = f't_T{[j]}')
t_D = {}
for j in range(c+2):
    t_D[j] = m.addVar(c, vtype = GRB.CONTINUOUS, name = f't_D{[j]}')

In [9]:
p = {}
for i in range(c+1):
    for j in range(1,c+1):
        if i!= j:
            p[i,j] = m.addVar(vtype = GRB.BINARY, name = f'p{[i,j]}')

In [10]:
u = {}
for i in range(1, c+2):
    u[i] = m.addVar(lb = 1, ub = c+2, vtype = GRB.INTEGER, name = f'u{[i]}')

In [11]:
# Objective
m.setObjective(t_T[c+1], GRB.MINIMIZE)

In [13]:
# Constraints
#for i in range(c+1):
#       for j in range(i+1,c+1):
#             if i != j:
for j in range(1,c+1):
    m.addConstr(quicksum(x[i,j] for i in range(c+1) if i != j) + quicksum(y[i,j,k] for i in range(c+1) for k in range(1,c+2) if (i,j,k) in F) == 1)

KeyError: (0, 1, 2)

In [14]:
m.addConstr(quicksum(x[0,j] for j in range(1,c+2)) == 1)
m.addConstr(quicksum(x[i,c+1] for i in range(c+1)) == 1)

<gurobi.Constr *Awaiting Model Update*>

In [15]:
for i in range(1,c+1):
    for j in range(1,c+2):
        if j != i:
            m.addConstr(u[i]-u[j] + 1 <= (c+2)*(1-x[i,j]))

In [16]:
for j in range(1,c+1):
    m.addConstr(quicksum(x[i,j] for i in range(c+1) if i != j) == quicksum(x[j,k] for k in range(1,c+2) if k != j))

In [17]:
for i in range(c+1):
    m.addConstr(quicksum(y[i,j,k] for j in Cdrones if j != i for k in range(1,c+2) if (i,j,k) in F) <= 1)

In [18]:
for k in range(1,c+2):
    m.addConstr(quicksum(y[i,j,k] for i in range(c+1) if i != k for j in Cdrones if (i,j,k) in F) <= 1)

In [19]:
for i in range(c+1):
    for j in Cdrones:
        for k in range(1, c+2):
                m.addConstr(2*y[i,j,k] <= quicksum(x[i,h] for h in range(1, c+2) if h != i) + quicksum(x[l,k] for l in range(c+1) if l != k))

In [20]:
for j in Cdrones:
    for k in range(1,c+2):
        if (0,j,k) in F:
            m.addConstr(y[0,j,k] <= quicksum(x[h,k] for h in range(c+1) if h != k))

In [21]:
for i in range(c+1):
    t_D[i] >= t_T[i] - M*(1-quicksum(y[i,j,k] for j in Cdrones if j != i for k in range(1,c+2) if (i,j,k) in F))

In [22]:
for i in range(c+1):
    t_T[i] >= t_D[i] - M*(1-quicksum(y[i,j,k] for j in Cdrones if j != i for k in range(1,c+2) if (i,j,k) in F))

In [23]:
for k in range(1,c+1):
    t_D[k] >= t_T[k] - M*(1-quicksum(y[i,j,k] for i in range(c+1) if i != k for j in Cdrones if (i,j,k) in F))

In [24]:
for k in range(1,c+1):
    t_T[k] >= t_D[k] - M*(1-quicksum(y[i,j,k] for i in range(c+1) if i != k for j in Cdrones if (i,j,k) in F))

In [25]:
for h in range(c+1):
    for k in range(1,c+2):
        if k != h:
            t_T[k] >= t_T[h] + tauTruck[h][k] + SL*(quicksum(y[h,l,m] for l in Cdrones if l!=h for m in range(1,c+2) if (h,l,m) in F)) + SR*(quicksum(y[i,j,k] for i in range(c+1) if i != k for j in Cdrones if (i,j,k) in F)) - M*(1-x[h,k])


In [26]:
for j in Cdrones:
    for i in range(c+1):
        if i != j:
            t_D[j] >= t_D[i] + tauDrone[i][j] + SL - M*(1 - quicksum(y[i,j,k] for k in range(1,c+2) if (i,j,k) in F))

In [27]:
for j in Cdrones:
    for k in range(1,c+2):
        if k != j:
            t_D[k] >= t_D[j] + tauDrone[j][k] + SR - M*(1 - quicksum(y[i,j,k] for i in range(c+1) if (i,j,k) in F))

In [28]:
for k in range(1,c+2):
    for i in range(c+1):
        t_D[k] - t_D[i] <= E + M*(1-quicksum(y[i,j,k] for j in Cdrones if (i,j,k) in F))

In [29]:
for i in range(1,c+1):
    for j in range(1,c+1):
        if j!= i:
            u[i] - u[j] >= 1-(c+2)*p[i,j]

In [30]:
for i in range(1,c+1):
    for j in range(1,c+1):
        if j!= i:
            u[i] - u[j] <= -1+(c+2)*(1-p[i,j])

In [31]:
for i in range(1,c+1):
    for j in range(1,c+1):
        if i!=j:
            p[i,j] + p[j,i] == 1

In [32]:
for i in range(c+1):
    for k in range(1,c+2):
        for l in range(1,c+1):
            if k != i and l != i and l != k:
                t_D[l] >= t_D[k] - M*(3-quicksum(y[i,j,k] for j in Cdrones if (i,j,k) in F if j != l) - quicksum(y[l,m,n] for m in Cdrones if m != i and m != k and m != l for n in range(1,c+2) if n != i and n != k and (l,m,n) in F) - p[i,l])

In [33]:
t_T[0] = 0

In [34]:
t_D[0] = 0

In [35]:
for j in range(1,c+1):
    p[0,j] = 1

In [36]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD E1-1200 APU with Radeon(tm) HD Graphics, instruction set [SSE2]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 122 rows, 148 columns and 962 nonzeros
Model fingerprint: 0xfecdf175
Variable types: 14 continuous, 134 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.14 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [ ]:
m.write("FSTSP.lp")

In [ ]:
y